In [ ]:
import pandas as pd
import numpy as np
import scipy.io
import cv2

from cmath import exp
from shutil import rmtree, move
from os import listdir, mkdir
from os.path import isfile, join, isdir
from scipy import signal

import os
import re

## Load the Data

In [ ]:
df_training = pd.read_csv('./bap-imgap-212022/train.csv')
print(df_training.head(), '\n')


df_test = pd.read_csv('./bap-imgap-212022/test.csv')
print(df_test.head())


def get_data_from_mat(train_file, test_file):
    train_mat = scipy.io.loadmat(train_file) 
    test_mat = scipy.io.loadmat(test_file) 

    train_np = np.array(train_mat['train_data']).transpose(2, 0, 1)
    test_np = np.array(test_mat['test_data']).transpose(2, 0, 1)

    print(train_np.shape)
    print(test_np.shape)

    return train_np, test_np

train_data, test_data = get_data_from_mat('./bap-imgap-212022/train_data.mat','./bap-imgap-212022/test_data.mat')

### 1. Create New Augmentation Directories

In [ ]:
N = len(train_data)
ages = []

path = './train_data/Original/'

if isdir(path):
    rmtree(path, ignore_errors = False, onerror = None)

mkdir(path)

for i in range(N):
    age = df_training.loc[df_training.index[i], 'age']

    if age not in ages:
        ages.append(age)

    if age:
        new_path = path + str(age) + '/'

        if not isdir(new_path):
            mkdir(new_path)

        cv2.imwrite(new_path + 
                    format(i, '05d') + '_' +
                    format(0, '05d') + '.png', train_data[i] * 255.0)

print(ages)

In [ ]:
DataAugTypes = ['Sharpness1', 'Sharpness2', 'GaussianBlur', 'EdgeDet1', 'EdgeDet2', 'EdgeDet3', 'EdgeDet4', 'EdgeDet5', 'EdgeDet6', 
'MotionBlur1', 'MotionBlur2', 'Box', 'Emboss1', 'Emboss2', 'FilterWithoutConv1', 'FilterWithoutConv2', 'FilterWithoutConv3', 
'FilterWithoutConv4', 'Deconvolution', 'Fourier1', 'Fourier2', 'HistEqualization1', 'HistEqualization2', 'GaussDiff1', 
'GaussDiff2', 'GaussDiff3', 'GaussDiff4', 'GaussDiff5', 'Harris', 'ShiTomasi', 'SIFT', 'ColorMap1', 'ColorMap2', 'ColorMap3',
'ColorMap4', 'ColorMap5', 'ColorMap6', 'ColorMap7', 'ColorMap8', 'ColorMap9', 'ColorMap10', 'ColorMap11', 'ColorMap12', 'ColorMap13', 
'ColorMap14', 'ColorMap15', 'ColorMap16', 'ColorMap17', 'ColorMap18', 'ColorMap19', 'ColorMap20', 'ColorMap21', 'ColorMap22', 
'Dilation', 'Closing', 'Thresholding', 'AdaptiveThresholding', 'Binarization']

path = './train_data'
original = path + '/Original/'

def createTypeFolders():
    for type in DataAugTypes:
        newpath = path + '/' + type

        if isdir(newpath):
            rmtree(newpath, ignore_errors = False, onerror = None)

        mkdir(newpath)

createTypeFolders()  

### 1.1 - Appliers

In [ ]:
def applyKernel(kernel, path, original):
    for a in ages:
        path_dir = path + str(a) + '/'

        if isdir(path_dir):
            rmtree(path_dir, ignore_errors = False, onerror = None)

        mkdir(path_dir)

        original_dir = original + str(a) + '/'
        onlyfiles = [f for f in listdir(original_dir) if isfile(join(original_dir, f))]

        for f in onlyfiles:
            
            img = cv2.imread(original_dir + f, cv2.IMREAD_COLOR)

            res = cv2.filter2D(src = img, ddepth = -1, kernel = kernel)
            
            cv2.imwrite(path_dir + f, res)



def applyDoubleKernel(kernel1, kernel2, path, original):
    for a in ages:
        path_dir = path + str(a) + '/'

        if isdir(path_dir):
            rmtree(path_dir, ignore_errors = False, onerror = None)

        mkdir(path_dir)

        original_dir = original + str(a) + '/'
        onlyfiles = [f for f in listdir(original_dir) if isfile(join(original_dir, f))]

        for f in onlyfiles:
            
            img = cv2.imread(original_dir + f, cv2.IMREAD_COLOR)

            resH = cv2.filter2D(src = img, ddepth = -1, kernel = kernel1)
            resV = cv2.filter2D(src = img, ddepth = -1, kernel = kernel2)

            res = resH + resV
            cv2.imwrite(path_dir + f, res)



def find_zero_crossings(res):
    res = np.asarray(res)
    res2 = [[np.abs(res[i][j] - res[i + 1][j])
                if (res[i][j] <= 0 and res[i + 1][j] > 0) or (res[i][j] >= 0 and res[i + 1][j] < 0)  else 0
                for j in range(1, res.shape[1] -1)] for i in range(1, res.shape[0] -1)]

    res3 =  [[np.abs(res[i][j]-res[i][j+1])
                if (res[i][j] <= 0 and res[i][j + 1] > 0) or (res[i][j] >= 0 and res[i][j + 1] < 0)  else 0
                for j in range(1, res.shape[1] - 1)] for i in range(1, res.shape[0] - 1)]

    res4 = np.asarray(res2) + np.asarray(res3)   
  
    return res4


def applyLaplacianAlternative(kernel1, kernel2, path, original):
    for a in ages:
        path_dir = path + str(a) + '/'

        if isdir(path_dir):
            rmtree(path_dir, ignore_errors = False, onerror = None)

        mkdir(path_dir)

        original_dir = original + str(a) + '/'
        onlyfiles = [f for f in listdir(original_dir) if isfile(join(original_dir, f))]

        for f in onlyfiles:
            
            img = cv2.imread(original_dir + f, cv2.IMREAD_COLOR)

            resX = signal.convolve2d(cv2.cvtColor(img, cv2.COLOR_BGR2GRAY), kernel1)
            resY = signal.convolve2d(cv2.cvtColor(img, cv2.COLOR_BGR2GRAY), kernel2)

            resT = resX + resY
            
            resZC = find_zero_crossings(resT)

            cv2.imwrite(path_dir + f, resZC)



def applyCanny(path, original):
    for a in ages:
        path_dir = path + str(a) + '/'

        if isdir(path_dir):
            rmtree(path_dir, ignore_errors = False, onerror = None)

        mkdir(path_dir)

        original_dir = original + str(a) + '/'
        onlyfiles = [f for f in listdir(original_dir) if isfile(join(original_dir, f))]

        for f in onlyfiles:
            
            img = cv2.imread(original_dir + f, cv2.IMREAD_COLOR)

            res = cv2.Canny(img, 70, 90)

            cv2.imwrite(path_dir + f, res)



def applyFilterWithoutConv(path, original, filter):
    for a in ages:
        path_dir = path + str(a) + '/'

        if isdir(path_dir):
            rmtree(path_dir, ignore_errors = False, onerror = None)

        mkdir(path_dir)

        original_dir = original + str(a) + '/'
        onlyfiles = [f for f in listdir(original_dir) if isfile(join(original_dir, f))]

        for f in onlyfiles:
            
            img = cv2.imread(original_dir + f, cv2.IMREAD_COLOR)

            if filter == 0:
                res = cv2.medianBlur(img, 3)

            elif filter == 1:
                res = cv2.GaussianBlur(img, (7, 7), -1)

            elif filter == 2:
                res = cv2.bilateralFilter(img, 3, 90, 90)
            
            elif filter == 3:
                gaussian = cv2.GaussianBlur(img, (7, 7), 0)
                unsharp_mask = cv2.addWeighted(img, 1.0, gaussian, -1.0, 0)
                res = cv2.medianBlur(unsharp_mask, 3)
            else:
                res = img

            cv2.imwrite(path_dir + f, res)



ANGLE = np.deg2rad(135)
D = 22
noise = 10 ** (-0.1 * 25)


def blur_edge(img, d = 31):
    h, w  = img.shape[:2]
    
    img_pad = cv2.copyMakeBorder(img, d, d, d, d, cv2.BORDER_WRAP)
    img_blur = cv2.GaussianBlur(img_pad, (2 * d + 1, 2 * d + 1), -1)[d:-d, d:-d]
    
    y, x = np.indices((h, w))
    dist = np.dstack([x, w - x - 1, y, h - y - 1]).min(-1)
    w = np.minimum(np.float32(dist) / d, 1.0)
    
    return img * w + img_blur * (1 - w)


def motion_kernel(angle, d, sz = 1):
    kern = np.ones((1, d), np.float32)
    c, s = np.cos(angle), np.sin(angle)
    
    A = np.float32([[c, -s, 0], [s, c, 0]])
    sz2 = sz // 2
    
    A[:, 2] = (sz2, sz2) - np.dot(A[:,:2], ((d - 1) * 0.5, 0))
    kern = cv2.warpAffine(kern, A, (sz, sz), flags = cv2.INTER_CUBIC)
    
    return kern


def applyDeconvolution(path, original):
    for a in ages:
        path_dir = path + str(a) + '/'

        if isdir(path_dir):
            rmtree(path_dir, ignore_errors = False, onerror = None)

        mkdir(path_dir)

        original_dir = original + str(a) + '/'
        onlyfiles = [f for f in listdir(original_dir) if isfile(join(original_dir, f))]

        for f in onlyfiles:
            img = cv2.imread(original_dir + f, cv2.IMREAD_GRAYSCALE)
            img = np.float32(img) / 255.0

            img_be = blur_edge(img)

            img_aux = img_be
            dft = cv2.dft(img_aux, flags = cv2.DFT_COMPLEX_OUTPUT)
            dft_shift = np.fft.fftshift(dft)
            
            psf = motion_kernel(ANGLE, D)
            
            psf /= psf.sum()
            psf_pad = np.zeros_like(img)
            kh, kw = psf.shape
            psf_pad[:kh, :kw] = psf
            PSF = cv2.dft(psf_pad, flags = cv2.DFT_COMPLEX_OUTPUT)

            PSF2 = (PSF ** 2).sum(-1)
            iPSF = PSF.conjugate() / (PSF2 + noise)[..., np.newaxis]
            RES = cv2.mulSpectrums(dft, iPSF, 0)
            res = cv2.idft(RES, flags = cv2.DFT_SCALE | cv2.DFT_REAL_OUTPUT)
            
            res = np.float32(res) * 255
            cv2.imwrite(path_dir + f, res)



def gauss(std, x, y):
    d = x ** 2 + y ** 2
    g =  exp(-d / (2 * std ** 2))

    return g

    
def gaussianLP(std, img_shape):
    return np.asarray([[gauss(std, x-img_shape[0]/2,y-img_shape[1]/2)  for y in range(img_shape[1])] for x in range(img_shape[0])])


def gaussianHP(std, img_shape):
    return np.asarray([[1 - gauss(std, x - img_shape[0] / 2, y - img_shape[1] / 2)  for y in range(img_shape[1])] for x in range(img_shape[0])])        


filter_strength = 8

def applyFourier(path, original, filter):
    for a in ages:
        path_dir = path + str(a) + '/'

        if isdir(path_dir):
            rmtree(path_dir, ignore_errors = False, onerror = None)

        mkdir(path_dir)

        original_dir = original + str(a) + '/'
        onlyfiles = [f for f in listdir(original_dir) if isfile(join(original_dir, f))]

        for f in onlyfiles:
            img = cv2.imread(original_dir + f, cv2.IMREAD_GRAYSCALE)

            image_grey_fourier = np.fft.fft2(img)

            if filter == 0:
                GaussianLP = gaussianLP(filter_strength, img.shape)
                filtered = np.fft.fftshift(image_grey_fourier) * GaussianLP
                LowPass = np.fft.ifftshift(filtered)
                res = np.fft.ifft2(LowPass)

            elif filter == 1:
                GaussianHP = gaussianHP(filter_strength, img.shape)
                filtered = np.fft.fftshift(image_grey_fourier) * GaussianHP
                HighPass = np.fft.ifftshift(filtered)
                res = np.fft.ifft2(HighPass)

            else: 
                res = img

            cv2.imwrite(path_dir + f, res.real)



def applyHistogramEqualization(path, original, filter):
    for a in ages:
        path_dir = path + str(a) + '/'

        if isdir(path_dir):
            rmtree(path_dir, ignore_errors = False, onerror = None)

        mkdir(path_dir)

        original_dir = original + str(a) + '/'
        onlyfiles = [f for f in listdir(original_dir) if isfile(join(original_dir, f))]

        for f in onlyfiles:
            img = cv2.imread(original_dir + f, cv2.IMREAD_GRAYSCALE)

            if filter == 0:
                res = cv2.equalizeHist(img)

            elif filter == 1:
                clahe = cv2.createCLAHE(clipLimit = 2.0, tileGridSize = (8, 8))
                res = clahe.apply(img)

            else:
                res = img

            cv2.imwrite(path_dir + f, res)



def gaussian_kernel(dimension):
    x = cv2.getGaussianKernel(dimension, -1)
    kernel = x.dot(x.T)

    return kernel


def applyGaussDiff(path, original, size):
    for a in ages:
        path_dir = path + str(a) + '/'

        if isdir(path_dir):
            rmtree(path_dir, ignore_errors = False, onerror = None)

        mkdir(path_dir)

        original_dir = original + str(a) + '/'
        onlyfiles = [f for f in listdir(original_dir) if isfile(join(original_dir, f))]

        for f in onlyfiles:
            img = cv2.imread(original_dir + f, cv2.IMREAD_GRAYSCALE)

            kernel1 = gaussian_kernel(size - 2)
            kernel2 = gaussian_kernel(size)

            res1 = cv2.filter2D(img, -1, kernel1)
            res2 = cv2.filter2D(img, -1, kernel2)

            res = res2 - res1

            cv2.imwrite(path_dir + f, res)



def applyHarrisCorners(path, original):
    for a in ages:
        path_dir = path + str(a) + '/'

        if isdir(path_dir):
            rmtree(path_dir, ignore_errors = False, onerror = None)

        mkdir(path_dir)

        original_dir = original + str(a) + '/'
        onlyfiles = [f for f in listdir(original_dir) if isfile(join(original_dir, f))]

        for f in onlyfiles:
            img = cv2.imread(original_dir + f, cv2.IMREAD_GRAYSCALE)
            dst = cv2.cornerHarris(img, 2, 11, 0.04)
            dst = cv2.dilate(dst, None)

            res = cv2.cvtColor(img, cv2.COLOR_GRAY2RGB)

            res[np.abs(dst) > 0.04 * dst.max()] = [0, 255, 0]

            cv2.imwrite(path_dir + f, res)



def applyShiTomasi(path, original):
    for a in ages:
        path_dir = path + str(a) + '/'

        if isdir(path_dir):
            rmtree(path_dir, ignore_errors = False, onerror = None)

        mkdir(path_dir)

        original_dir = original + str(a) + '/'
        onlyfiles = [f for f in listdir(original_dir) if isfile(join(original_dir, f))]

        for f in onlyfiles:
            img = cv2.imread(original_dir + f, cv2.IMREAD_GRAYSCALE)

            corners = cv2.goodFeaturesToTrack(img, 2500, 0.01, 10)

            corners = np.int0(corners)

            res = cv2.cvtColor(img, cv2.COLOR_GRAY2RGB)

            for i in corners:
                x, y = i.ravel()
                cv2.circle(res, (x, y), 1, 255, -1)

            cv2.imwrite(path_dir + f, res)



def applySIFT(path, original):
    for a in ages:
        path_dir = path + str(a) + '/'

        if isdir(path_dir):
            rmtree(path_dir, ignore_errors = False, onerror = None)

        mkdir(path_dir)

        original_dir = original + str(a) + '/'
        onlyfiles = [f for f in listdir(original_dir) if isfile(join(original_dir, f))]

        for f in onlyfiles:
            img = cv2.imread(original_dir + f, cv2.IMREAD_GRAYSCALE)
            
            sift = cv2.SIFT_create()
            kp = sift.detect(img, None)

            res = cv2.drawKeypoints(img, kp, img, flags = cv2.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS)

            cv2.imwrite(path_dir + f, res)



def applyColorMap(path, original, map):
    for a in ages:
        path_dir = path + str(a) + '/'

        if isdir(path_dir):
            rmtree(path_dir, ignore_errors = False, onerror = None)

        mkdir(path_dir)

        original_dir = original + str(a) + '/'
        onlyfiles = [f for f in listdir(original_dir) if isfile(join(original_dir, f))]

        for f in onlyfiles:
            img = cv2.imread(original_dir + f, cv2.IMREAD_COLOR)

            res = cv2.applyColorMap(img, map)

            cv2.imwrite(path_dir + f, res)



def applyTransform(path, original, flag):
    for a in ages:
        path_dir = path + str(a) + '/'

        if isdir(path_dir):
            rmtree(path_dir, ignore_errors = False, onerror = None)

        mkdir(path_dir)

        original_dir = original + str(a) + '/'
        onlyfiles = [f for f in listdir(original_dir) if isfile(join(original_dir, f))]

        for f in onlyfiles:
            img = cv2.imread(original_dir + f, cv2.IMREAD_COLOR)

            if flag == 0:
                kernel = np.ones((3, 3), np.uint8)
                res = cv2.morphologyEx(img, cv2.MORPH_CLOSE, kernel)

            elif flag == 1:
                kernel = np.ones((3, 3), np.uint8)
                res = cv2.dilate(img, kernel, iterations = 1)

            else:
                res = img

            cv2.imwrite(path_dir + f, res)



def applyThresholding(path, original, flag):
    for a in ages:
        path_dir = path + str(a) + '/'

        if isdir(path_dir):
            rmtree(path_dir, ignore_errors = False, onerror = None)

        mkdir(path_dir)

        original_dir = original + str(a) + '/'
        onlyfiles = [f for f in listdir(original_dir) if isfile(join(original_dir, f))]

        for f in onlyfiles:
            img = cv2.imread(original_dir + f, 0)

            if flag == 0:
                img = cv2.GaussianBlur(img, (3, 3), 0)
                _, res = cv2.threshold(img, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)

            elif flag == 1: 
                img = cv2.GaussianBlur(img, (3, 3), 0)
                res = cv2.adaptiveThreshold(img, 255, cv2.ADAPTIVE_THRESH_MEAN_C, cv2.THRESH_BINARY, 3, 5)

            elif flag == 2:
                _, res = cv2.threshold(img, 0, 255, cv2.THRESH_BINARY)

            else:
                res = img

            cv2.imwrite(path_dir + f, res)

### 2. Sharpness Filters

In [ ]:
kernel1 = np.array([[0, -1, 0],
                    [-1, 5,-1],
                    [0, -1, 0]])

kernel2 = np.array([[-1, -1, -1],
                    [-1,  9, -1],
                    [-1, -1, -1]])


kernel1 = kernel1 / np.sum(kernel1)

kernel2 = kernel2 / np.sum(kernel2)

path1 = './train_data/Sharpness1/'
path2 = './train_data/Sharpness2/'

applyKernel(kernel1, path1, original)
applyKernel(kernel2, path2, original)

### 4. Gaussian Blur

In [ ]:
def gaussian_kernel(dimension):
    x = cv2.getGaussianKernel(dimension, -1)
    kernel = x.dot(x.T)
    return kernel

kernel = gaussian_kernel(3)

path = './train_data/GaussianBlur/'

applyKernel(kernel, path, original)

### 5. Edge Detection

In [ ]:
# Simple Laplacian Approximations
kernel1 = np.array([[-1, -1, -1],
                    [-1, 8, -1],
                    [-1, -1, -1]])

kernel2 = np.array([[0, 1, 0],
                    [1, -4, 1],
                    [0, 1, 0]])


# Prewit
kernel3H = np.array([[-1, -1, -1],
                     [0,  0,  0],
                     [1,  1,  1]])

kernel3V = np.array([[-1, 0, 1],
                     [-1, 0, 1],
                     [-1, 0, 1]])


# Scharr
kernel4H = np.array([[-3, -10, -3],
                     [0,   0,  0],
                     [3,  10,  3]])

kernel4V = np.array([[-3, 0,  3],
                     [-10, 0, 10],
                     [-3, 0,  3]])


# Laplacian Alternative
kernel5X = np.array([[-1, 2, -1],
                     [-2, 4, -2],
                     [-1, 2, -1]])

kernel5Y = np.array([[-1, -2, -1],
                     [2,  4,  2],
                     [-1, -2, -1]])


path1 = './train_data/EdgeDet1/'
path2 = './train_data/EdgeDet2/'
path3 = './train_data/EdgeDet3/'
path4 = './train_data/EdgeDet4/'
path5 = './train_data/EdgeDet5/'
path6 = './train_data/EdgeDet6/'

applyKernel(kernel1, path1, original)
applyKernel(kernel2, path2, original)
applyDoubleKernel(kernel3H, kernel3V, path3, original)
applyDoubleKernel(kernel4H, kernel4V, path4, original)
applyLaplacianAlternative(kernel5X, kernel5Y, path5, original)
applyCanny(path6, original)

### 6. Motion Blur

In [ ]:
kernel1 = np.array([[0, 0, 0, 0, 0, 0, 1],
                    [0, 0, 0, 0, 0, 1, 0],
                    [0, 0, 0, 0, 1, 0, 0],
                    [0, 0, 0, 1, 0, 0, 0],
                    [0, 0, 1, 0, 0, 0, 0],
                    [0, 1, 0, 0, 0, 0, 0],
                    [1, 0, 0, 0, 0, 0, 0]])

kernel2 = np.array([[0, 0, 0, 0, 0, 0, 0],
                    [0, 0, 0, 0, 0, 0, 0],
                    [0, 0, 0, 0, 1, 1, 1],
                    [0, 0, 1, 1, 0, 0, 0],
                    [1, 1, 0, 0, 0, 0, 0],
                    [0, 0, 0, 0, 0, 0, 0],
                    [0, 0, 0, 0, 0, 0, 0]])

kernel1 = kernel1 / np.sum(kernel1)
kernel2 = kernel2 / np.sum(kernel2)

path1 = './train_data/MotionBlur1/'
path2 = './train_data/MotionBlur2/'

applyKernel(kernel1, path1, original)
applyKernel(kernel2, path2, original)

### 7. Box filter

In [ ]:
dim = 7
kernel = np.ones((dim, dim))
kernel = kernel / np.sum(kernel)

path = './train_data/Box/'

applyKernel(kernel, path, original)

### 8. Emboss

In [ ]:
kernel1 = np.array([[-2, -1, 0],
                    [-1,  1, 1],
                    [ 0,  1, 2]])

kernel2 = np.array([[0,  1, 2],
                    [-1,  1, 1],
                    [-2, -1, 0]])

kernel1 = kernel1 / np.sum(kernel1)

kernel2 = kernel2 / np.sum(kernel2)

path1 = './train_data/Emboss1/'
path2 = './train_data/Emboss2/'

applyKernel(kernel1, path1, original)
applyKernel(kernel2, path2, original)

### 9. Filters Without Convolution

In [ ]:
path1 = './train_data/FilterWithoutConv1/'
path2 = './train_data/FilterWithoutConv2/'
path3 = './train_data/FilterWithoutConv3/'
path4 = './train_data/FilterWithoutConv4/'


# Median Blur
applyFilterWithoutConv(path1, original, filter = 0)

# Gaussian Blur
applyFilterWithoutConv(path2, original, filter = 1)

# Bilateral Filter
applyFilterWithoutConv(path3, original, filter = 2)

# Unsharp Mask
applyFilterWithoutConv(path4, original, filter = 3)

### 10. Deconvolution

In [ ]:
path = './train_data/Deconvolution/'

applyDeconvolution(path, original)

### 11. Fourier Transform

In [ ]:
path1 = './train_data/Fourier1/'
path2 = './train_data/Fourier2/'

applyFourier(path1, original, 0)
applyFourier(path2, original, 1)

### 12. Histogram Equalization

In [ ]:
path1 = './train_data/HistEqualization1/'
path2 = './train_data/HistEqualization2/'

# Normal Equalization
applyHistogramEqualization(path1, original, 0)

# CLAHE
applyHistogramEqualization(path2, original, 1)

### 13. Difference of Gaussians

In [ ]:
path1 = './train_data/GaussDiff1/'
path2 = './train_data/GaussDiff2/'
path3 = './train_data/GaussDiff3/'
path4 = './train_data/GaussDiff4/'
path5 = './train_data/GaussDiff5/'

applyGaussDiff(path1, original, 3)
applyGaussDiff(path2, original, 5)
applyGaussDiff(path3, original, 7)
applyGaussDiff(path4, original, 9)
applyGaussDiff(path5, original, 11)

### 14. Feature Detection Algorithms

In [ ]:
path1 = './train_data/Harris/'
path2 = './train_data/ShiTomasi/'
path3 = './train_data/SIFT/'

# Harris Corners
applyHarrisCorners(path1, original)

# Shi-Tomasi
applyShiTomasi(path2, original)

# SIFT
applySIFT(path3, original)

### 15. Color Mapping

In [ ]:
for i in range(0, 22):
    path = f'./train_data/ColorMap{i + 1}/'
    applyColorMap(path, original, i)

### 16. Closing and Dilation

In [ ]:
path1 = './train_data/Closing/'
path2 = './train_data/Dilation/'

applyTransform(path1, original, 0)
applyTransform(path2, original, 1)

### 17. Thresholding, Adaptive Thresholding, And Binarization

In [ ]:
path1 = './train_data/Thresholding/'
path2 = './train_data/AdaptiveThresholding/'
path3 = './train_data/Binarization/'

applyThresholding(path1, original, 0)
applyThresholding(path2, original, 1)
applyThresholding(path3, original, 2)

## Validation Set Creation

In [ ]:
def createValSet(path):
    subfolders = [f.path for f in os.scandir(path) if f.is_dir()]
    
    for sf in subfolders:
        new_path = re.sub(r'train_data', r'val_data', sf)

        if isdir(new_path):
            rmtree(new_path, ignore_errors = False, onerror = None)

        mkdir(new_path)

        for a in ages:
            path_dir = sf + '/' + str(a) + '/'
            new_path_dir = new_path + '/' + str(a) + '/'

            onlyfiles = [f for f in listdir(path_dir) if isfile(join(path_dir, f))]
            
            length = len(onlyfiles)

            if length == 1:
                train_length = 1
                val_length = 0
            
            elif length > 1 and length < 5:
                train_length = length - 1
                val_length = 1  

            else:
                train_length = int(length * 0.8)
                val_length = length - train_length

            valfiles = np.random.choice(onlyfiles, size = val_length, replace = False)

            if isdir(new_path_dir):
                rmtree(new_path_dir, ignore_errors = False, onerror = None)

            mkdir(new_path_dir)

            for v in valfiles:
                move(path_dir + v, new_path_dir + v)


createValSet('./train_data')